<a href="https://colab.research.google.com/github/hellomada/RayField-Systems-Internship-Week-2/blob/notebooks/ai_model_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""# Day 1 – ML Problem Setup & First Model

## Problem Statement
We aim to **forecast AC power output** (`ac_power`) for a solar plant using features such as hour of day, direct current power, and energy yields.

---

### Input Features:
- `hour`
- `dc_power`
- `daily_yield`
- `total_yield`

### Target Output:
- `ac_power`

---

## Baseline Model
We'll use `LinearRegression` from scikit-learn to predict continuous AC power output.
"""


SyntaxError: unterminated string literal (detected at line 20) (ipython-input-1319333550.py, line 20)

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score

# Load the data
df = pd.read_csv("cleaned_data.csv")
df = df.dropna()  # Drop missing values
df.head()


,date_time,plant_id,source_key,dc_power,ac_power,daily_yield,total_yield,hour
0,2020-05-15 00:00:00,4136001,4UPUqMRk7TRMgml,0.0,0.0,9425.000000,2429011.0,0
1,2020-05-15 00:00:00,4136001,xoJJ8DcxJEcupym,0.0,0.0,0.000000,209143593.0,0
2,2020-05-15 00:00:00,4136001,xMbIugepa2P7lBB,0.0,0.0,9166.000000,106656621.0,0
3,2020-05-15 00:00:00,4136001,vOuJvMaM2sgwLmb,0.0,0.0,0.000000,2211962.0,0
4,2020-05-15 00:00:00,4136001,rrq4fwE8jgrTyWY,0.0,0.0,280.214286,120964108.0,0


In [38]:
df.describe()

,plant_id,dc_power,ac_power,daily_yield,total_yield,hour
count,67698.0,67698.000000,67698.000000,67698.000000,6.769800e+04,67698.000000
mean,4136001.0,246.701961,241.277825,3294.890295,6.589448e+08,11.530208
std,0.0,370.569597,362.112118,2919.448386,7.296678e+08,6.922127
min,4136001.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,4136001.0,0.000000,0.000000,272.750000,1.996494e+07,6.000000
50%,4136001.0,0.000000,0.000000,2911.000000,2.826276e+08,12.000000
75%,4136001.0,446.591667,438.215000,5534.000000,1.348495e+09,18.000000
max,4136001.0,1420.933333,1385.420000,9873.000000,2.247916e+09,23.000000


In [9]:
X = df[['hour', 'dc_power', 'daily_yield', 'total_yield']]
Y = df[['ac_power']]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [15]:
# Create the LR model
model = LinearRegression()

# Train the model
model.fit(x_train, y_train)

preds = model.predict(x_test)
mse = mean_squared_error(y_test, preds)

print(f"MSE: {mse}")


MSE: 0.825586902468569


In [20]:
# Create random forest regressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)

# Predict
preds = rf_model.predict(x_test)
mse = mean_squared_error(y_test, preds)

print(f"MSE: {mse}")

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


MSE: 0.27548887657901705


In [34]:


from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression

y_train_ravel = y_train.values.ravel()
y_test_ravel = y_test.values.ravel()

GBR = GradientBoostingRegressor(n_estimators=100, random_state=42)
GBR.fit(x_train, y_train_ravel)

RFR = RandomForestRegressor(n_estimators=100, random_state=42)
RFR.fit(x_train, y_train_ravel)

ADR = AdaBoostRegressor(n_estimators=100, random_state=42)
ADR.fit(x_train, y_train_ravel)

LR= LinearRegression()
LR.fit(x_train, y_train_ravel)

LinearRegression()

In [36]:
er = VotingRegressor([
    ("GBR", GBR),
     ("RFR", RFR),
      ("ADR", ADR),
       ("LR", LR)
       ])

er.fit(x_train, y_train_ravel)

VotingRegressor(estimators=[('GBR', GradientBoostingRegressor(random_state=42)),
                            ('RFR', RandomForestRegressor(random_state=42)),
                            ('ADR',
                             AdaBoostRegressor(n_estimators=100,
                                               random_state=42)),
                            ('LR', LinearRegression())])

In [37]:
pred = er.predict(x_test)
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, pred)
r2 = r2_score(y_test, pred)

print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")


Mean Squared Error: 17.731210542650896
R2 Score: 0.9998633399645259
